In [ ]:
#importing packages 
import numpy as np
from pythermalcomfort.models import pmv_ppd_iso
import matplotlib.pyplot as plt

#Imputs needed for function to find thermal comfort 
def simulate_thermal_comfort(
        T_outdoor=80,
        T_room_initial=80,
        tau=60,
        rh=50,
        met=1.0,
        clo=None,
        wind_speeds=[0, 0.1, 0.5, 1.5],  # m/s,         # air speeds
        radiant_offsets=[-0.2, 0, 0.2],  # radiant offsets in °C
        initial_pmv=2.0,
        season=None,
        total_minutes=720,
        device="fan",
        baseline="AC",
        city="default",
        T_setpoint=None
):
    # --- Device effects setup ---
    device_effects = {
        "AC": {"temp_delta": -5, "rh_delta": -5, "air_speed": 0.1, "clo": 0.5},
        "fan": {"temp_delta": 0, "rh_delta": 0, "air_speed": 0.8, "clo": 0.5},
        "heater": {"temp_delta": +5, "rh_delta": -5, "air_speed": 0.1, "clo": 1.0},
        "space_heater": {"temp_delta": +3, "rh_delta": -3, "air_speed": 0.2, "clo": 1.0},
        "dehumidifier": {"temp_delta": 0, "rh_delta": -15, "air_speed": 0, "clo": clo},
        "window": {"temp_delta": -0.5, "rh_delta": 0, "air_speed": 0.3, "clo": clo},
        "none": {"temp_delta": 0, "rh_delta": 0, "air_speed": 0, "clo": clo},
        "add_clothing": {"temp_delta": 0, "rh_delta": 0, "air_speed": 0, "clo": 1.2},
        "remove_clothing": {"temp_delta": 0, "rh_delta": 0, "air_speed": 0, "clo": 0.4},
        "heat": {"temp_delta": +5, "rh_delta": -5, "air_speed": 0.1, "clo": 1.0},  # Added "heat" device
    }
    # --- CLO setup based on season ---
    if clo is None:
        if season == 'summer':
            clo = 0.5
        elif season == 'winter':
            clo = 1.0
        else:
            clo = 0.5

    # --- CLO override from baseline if defined ---
    baseline_clo_effect = device_effects.get(baseline, {}).get("clo", clo)
    if baseline_clo_effect is not None:
        clo = baseline_clo_effect

    # --- Combine multiple devices if list ---
    if isinstance(device, list):
        selected_device = {"temp_delta": 0, "rh_delta": 0, "air_speed": 0, "clo": clo}
        for d in device:
            effect = device_effects.get(d, device_effects["none"])
            selected_device["temp_delta"] += effect.get("temp_delta", 0)
            selected_device["rh_delta"] += effect.get("rh_delta", 0)
            selected_device["air_speed"] = max(selected_device["air_speed"], effect.get("air_speed", 0))
            selected_device["clo"] = effect.get("clo", selected_device["clo"])  # override with latest
    else:
        selected_device = device_effects.get(device, device_effects["none"])

    # Apply device air speed effect to wind speeds if using a device that changes air movement
    if selected_device["air_speed"] > 0:
        # Add device-specific air speed to wind speeds list if not already included
        if selected_device["air_speed"] not in wind_speeds:
            wind_speeds = list(wind_speeds) + [selected_device["air_speed"]]

    # Adjust relative humidity based on device effect
    adjusted_rh = max(min(rh + selected_device["rh_delta"], 95), 5)  # Keep RH between 5% and 95%

    # --- Setpoint override or PMV-based delta ---
    if T_setpoint is not None:
        T_target_F = T_setpoint
    else:
        # Calculate delta based on initial PMV
        if initial_pmv >= 2:
            delta = -5 if initial_pmv >= 3 else -3
        elif initial_pmv <= -2:
            delta = +5 if initial_pmv <= -3 else +3
        else:
            delta = 0
        # Apply delta to get target temperature
        T_target_F = T_room_initial + delta

    # Apply device temperature effect
    device_temp_effect_F = selected_device["temp_delta"]  # Temperature change in °F
    T_target_effective_F = T_target_F + device_temp_effect_F

    # Convert temperatures to Celsius
    T_target_C = (T_target_effective_F - 32) * 5 / 9
    T_room_initial_C = (T_room_initial - 32) * 5 / 9

    # --- Time & result storage ---
    minutes = np.arange(0, total_minutes, 1)
    results = {}
    time_to_comfort = None  # Initialize time_to_comfort variable
    temp_curves = {}

#  --- Wind speeds  --- 
    for v in wind_speeds:
        for tr_offset in radiant_offsets:
            temps = T_target_C + (T_room_initial_C - T_target_C) * np.exp(-minutes / tau)

            if v == wind_speeds[0] and tr_offset == radiant_offsets[0]:
                temp_curves["Room Temperature"] = [(t * 9 / 5) + 32 for t in temps]
                temp_curves["Effective Temperature"] = [(t * 9 / 5) + 32 + device_temp_effect_F for t in temps]

            pmvs = []
            for ta in temps:
                tr = ta + tr_offset
                effective_v = max(v, selected_device["air_speed"]) if device != "none" else v
                pmv_result = pmv_ppd_iso(tdb=ta, tr=tr, vr=effective_v, rh=adjusted_rh, met=met, clo=clo)
                pmvs.append(pmv_result['pmv'])

            
            if v == selected_device["air_speed"] and device != "none":
                device_label = ", ".join(device) if isinstance(device, list) else device
                key = f"{device_label}: v={v}m/s, tr_offset={tr_offset}°C"
            else:
                key = f"v={v}m/s, tr_offset={tr_offset}°C"

            results[key] = pmvs

            for i, pmv in enumerate(pmvs):
                if -0.2 <= pmv <= 0.2:
                    if time_to_comfort is None or i < time_to_comfort:
                        time_to_comfort = i
                    break

    # --- Calculate Energy, CO2, Cost ---
    power_kW = {
        "AC": 2.0,
        "window_AC": 1.0,
        "fan": 0.06,
        "heater": 1.0,
        "heat": 1.0,  # Added "heat" power consumption
        "space_heater": 1.5,
        "dehumidifier": 0.3,
        "window": 0.0,
        "none": 0.0,
        "add_clothing": 0.0,
        "remove_clothing": 0.0
    }

    emissions_kg_per_kWh = {
        "pittsburgh": 0.331,
        "new york": 0.202,
        "seattle": 0.259,
        "default": 0.33
    }

#Using Pittsburgh cost per kWhr  
    cost_per_kWh = 0.17
    hours = (time_to_comfort or 0) / 60  # Use '0' if time_to_comfort is None

    power_baseline = power_kW.get(baseline, 0)
    if isinstance(device, list):
        power_action = sum([power_kW.get(d, 0) for d in device])
    else:
        power_action = power_kW.get(device, 0)
    energy_diff = (power_baseline - power_action) * hours
    emission_factor = emissions_kg_per_kWh.get(city.lower(), emissions_kg_per_kWh["default"])

    carbon_kg = energy_diff * emission_factor
    cost_usd = energy_diff * cost_per_kWh

    # --- Print Summary ---
    if time_to_comfort is not None:
        print(f"\nPMV ≈ 0 reached in {time_to_comfort} min ({hours:.2f} hrs)")
        print(f"Energy saved: {energy_diff:.2f} kWh")
        print(f"Carbon saved: {carbon_kg:.2f} kg CO₂")
        print(f"Cost saved: ${cost_usd:.2f}")
    else:
        print(f"\nComfort not reached within {total_minutes / 60} hours.")

    # --- Plotting ---
    # --- Time to reach target room and effective temperature ---
    room_temp_list = temp_curves.get("Room Temperature", [])
    effective_temp_list = temp_curves.get("Effective Temperature", [])
    
    time_to_target_room = next((i for i, t in enumerate(room_temp_list) if abs(t - T_target_F) <= 0.2), None)
    time_to_target_effective = next((i for i, t in enumerate(effective_temp_list) if abs(t - T_target_F) <= 0.2), None)
    
    if time_to_target_room is not None:
        print(f"Room temperature reaches target ({T_target_F}°F) in {time_to_target_room} min ({time_to_target_room / 60:.2f} hrs)")
    else:
        print(f"Room temperature does not reach target ({T_target_F}°F) within {total_minutes / 60} hours.")
    
    if time_to_target_effective is not None:
        print(f"Effective temperature reaches target ({T_target_F}°F) in {time_to_target_effective} min ({time_to_target_effective / 60:.2f} hrs)")
    else:
        print(f"Effective temperature does not reach target ({T_target_F}°F) within {total_minutes / 60} hours.")

    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 15))

    # Temperature plot
    ax1.plot(minutes / 60, temp_curves["Room Temperature"], label="Room Temperature", linewidth=2)
    if "Effective Temperature" in temp_curves:
        ax1.plot(minutes / 60, temp_curves["Effective Temperature"], label=f"Effective Temperature ({device})",
                linewidth=2, linestyle='--')
    ax1.axhline(T_room_initial, color='gray', linestyle='--', label=f'Initial Temp: {T_room_initial}°F')
    ax1.axhline(T_target_F, color='blue', linestyle='--', label=f'Target Temp: {T_target_F}°F')
    ax1.set_xlabel('Hours')
    ax1.set_ylabel('Temperature (°F)')
    ax1.legend()
    ax1.grid(True)

    # Absolute PMV plot
    for label, pmvs in results.items():
        ax2.plot(minutes / 60, pmvs, label=label)
    ax2.axhline(0, color='gray', linestyle='--', label='Neutral (PMV = 0)')
    ax2.axhline(0.5, color='lightgray', linestyle=':', label='Slightly warm (+0.5)')
    ax2.axhline(-0.5, color='lightgray', linestyle=':', label='Slightly cool (-0.5)')
    ax2.set_xlabel('Hours')
    ax2.set_ylabel('PMV')
    ax2.set_title('Absolute PMV Value Over Time')
    ax2.legend()
    ax2.grid(True)



    # Calculate time to reach different PMV thresholds
    times_to_pmv_thresholds = {}
    baseline_pmvs = results.get(f"v={wind_speeds[0]}m/s, tr_offset={radiant_offsets[0]}°C", [])

    if baseline_pmvs:
        initial_pmv = baseline_pmvs[0]
        thresholds = [0.8, 0.5, 0.2, 0, -0.2, -0.5, -0.8]

        for threshold in thresholds:
            # For cooling (initial PMV > threshold)
            if initial_pmv > threshold:
                for i, pmv in enumerate(baseline_pmvs):
                    if pmv <= threshold:
                        times_to_pmv_thresholds[f"Time to PMV {threshold}"] = i
                        break
            # For heating (initial PMV < threshold)
            elif initial_pmv < threshold:
                for i, pmv in enumerate(baseline_pmvs):
                    if pmv >= threshold:
                        times_to_pmv_thresholds[f"Time to PMV {threshold}"] = i
                        break

    return {
        "fig": fig,
        "time_minutes": time_to_comfort,
        "energy_kWh": round(energy_diff, 2),
        "carbon_kg": round(carbon_kg, 2),
        "cost_usd": round(cost_usd, 2),
        "pmv_thresholds": times_to_pmv_thresholds
    }


# === Usage ===
def run_simulations():
    scenarios = [
        {
            "name": "Case 1a: Summer, very uncomfortable (very hot), outside is 90 F "
                    "(A/C setpoint to 72 F), fast tau",
            "params": {
                "T_outdoor": 90,  #outside temp 
                "T_room_initial": 89,  #start temperature of the room 
                "tau": 15,  #Tau in hours 
                "initial_pmv": 3.0,  #PMV level of the user 
                "season": "summer",  #current season 
                "device": ["fan"],  #cooling action
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",  #City in the case 
                "T_setpoint": 72  #Set point of the AC 
            }
        },
        {
            "name": "Case 1b: Summer, very uncomfortable (very hot), outside is 90 F (A/C setpoint to 72 F)",
            "params": {
                "T_outdoor": 90,
                "T_room_initial": 89,
                "tau": 32,
                "initial_pmv": 3.0,
                "season": "summer",
                "device": ["fan"],  # cooling action
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 72
            }
        },
        {
            "name": "Case 1c: Summer, very uncomfortable (very hot), outside is 90 F (A/C setpoint to 72 F)",
            "params": {
                "T_outdoor": 90,
                "T_room_initial": 89,
                "tau": 57,
                "initial_pmv": 3.0,
                "season": "summer",
                "device": ["fan"],  # cooling action
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 72
            }
        },
        {
            "name": "Case 2a: Winter, very uncomfortable (very cold), outside 20 F (Heat setpoint 73F)",
            "params": {
                "T_outdoor": 20,
                "T_room_initial": 67,
                "tau": 15,
                "initial_pmv": -3.0,
                "season": "winter",
                "device": ["space_heater", "add_clothing"],  #
                "baseline": "heat",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 73
            }
        },
        {
            "name": "Case 2b: Winter, very uncomfortable (very cold), outside 20 F (Heat setpoint 73F)",
            "params": {
                "T_outdoor": 20,
                "T_room_initial": 67,
                "tau": 32,
                "initial_pmv": -3.0,
                "season": "winter",
                "device": ["space_heater", "add_clothing"],  #
                "baseline": "heat",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 73
            }
        },
        {
            "name": "Case 2c: Winter, very uncomfortable (very cold), outside 20 F (Heat setpoint 73F)",
            "params": {
                "T_outdoor": 20,
                "T_room_initial": 67,
                "tau": 57,
                "initial_pmv": -3.0,
                "season": "winter",
                "device": ["space_heater", "add_clothing"],  #
                "baseline": "heat",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 73
            }
        },
        {
            "name": "Case 3a: Summer, medium uncomfortable (cold), outside is 75 F (stop A/C)",
            "params": {
                "T_outdoor": 75,
                "T_room_initial": 70,
                "tau": 6,
                "initial_pmv": -2.0,
                "season": "summer",
                "device": ["add_clothing","heat"],
                "baseline": "heat",
                "city": "pittsburgh",
                "T_setpoint": 75
            }
        },
        {
            "name": "Case 3b: Summer, medium uncomfortable (cold), outside is 73 F (stop A/C)",
            "params": {
                "T_outdoor": 75,
                "T_room_initial": 70,
                "tau": 24,
                "initial_pmv": -2.0,
                "season": "summer",
                "device": ["add_clothing", "none"],
                "baseline": "heat",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 75
            }
        },
        {
            "name": "Case 3c: Summer, medium uncomfortable (cold), outside is 73 F (stop A/C)",
            "params": {
                "T_outdoor": 75,
                "T_room_initial": 70,
                "tau": 60,
                "initial_pmv": -2.0,
                "season": "summer",
                "device": ["add_clothing", "none"],
                "baseline": "heat",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 75
            }
        },
        {
            "name": "Case 4a: Winter, medium uncomfortable (warm/hot), outside is 60 F (heat setpoint 68 F)",
            "params": {
                "T_outdoor": 60,
                "T_room_initial": 76,
                "tau": 4,
                "initial_pmv": 2.0,
                "season": "winter",
                "device": ["fan", "window"],
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 68
            }
        },
        {
            "name": "Case 4b: Winter, medium uncomfortable (warm/hot), outside is 60 F (heat setpoint 68 F)",
            "params": {
                "T_outdoor": 60,
                "T_room_initial": 76,
                "tau": 30,
                "initial_pmv": 2.0,
                "season": "winter",
                "device": ["fan", "window"],
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 68
            }
        },
        {
            "name": "Case 4c: Winter, medium uncomfortable (warm/hot), outside is 60 F (heat setpoint 68 F)",
            "params": {
                "T_outdoor": 60,
                "T_room_initial": 76,
                "tau": 76,
                "initial_pmv": 2.0,
                "season": "winter",
                "device": ["fan", "window"],  #
                "baseline": "AC",  # baseline without energy-saving action
                "city": "pittsburgh",
                "T_setpoint": 68
            }
        }

    ]

    for scenario in scenarios:
        print(f"\n=== {scenario['name']} ===")
        # Run simulation with the selected device
        device_result = simulate_thermal_comfort(**scenario["params"])

        # Get device name for clear reporting
        device_name = scenario["params"]["device"]
        baseline_name = scenario["params"]["baseline"]

        print(f"Device: {device_name} vs Baseline: {baseline_name}")
        print(f"Energy Saved: {device_result['energy_kWh']} kWh")
        print(f"Carbon Saved: {device_result['carbon_kg']} kg CO₂")
        print(f"Cost Saved: ${device_result['cost_usd']}")

        # Display PMV threshold timing information
        if device_result['pmv_thresholds']:
            print("\nTime to reach PMV thresholds:")
            for threshold, minutes in device_result['pmv_thresholds'].items():
                print(f"  {threshold}: {minutes} minutes ({minutes / 60:.2f} hours)")

        # Now run again with the baseline device for comparison
        if scenario["params"]["device"] != scenario["params"]["baseline"]:
            print("\nRunning comparison with baseline device...")
            comparison_params = scenario["params"].copy()
            comparison_params["device"] = scenario["params"]["baseline"]
            baseline_result = simulate_thermal_comfort(**comparison_params)

            # Compare comfort achievement times
            if device_result['time_minutes'] is not None and baseline_result['time_minutes'] is not None:
                comfort_diff = device_result['time_minutes'] - baseline_result['time_minutes']
                if comfort_diff > 0:
                    print(
                        f"The {baseline_name} reaches neutral comfort {comfort_diff} minutes faster than the {device_name}")
                elif comfort_diff < 0:
                    print(
                        f"The {device_name} reaches neutral comfort {-comfort_diff} minutes faster than the {baseline_name}")
                else:
                    print(f"Both devices reach neutral comfort at the same time")

        device_result["fig"].suptitle(f"Thermal Comfort Analysis: {scenario['name']}", fontsize=14)
        plt.show()


if __name__ == "__main__":
    run_simulations()